Importing Libraries

In [23]:
import pandas as pd
from datetime import datetime

Loading Data

In [24]:
def load_data(file_path):
    """Load data from an Excel file and drop duplicate values."""
    df = pd.read_excel(file_path)
    df.drop_duplicates(subset=['Email'], keep='first', inplace=True)
    return df
file_path = 'Final Lead Data.xlsx'
df = load_data(file_path)

Data Inspection

In [25]:
def inspect_data(df):
    """Inspect data structure and summary statistics."""
    print("Data Information:")
    print(df.info())
    print("\nData Summary Statistics:")
    print(df.describe())
    print("\nFirst 5 Rows of Data:")
    print(df.head())

inspect_data(df)

Data Information:
<class 'pandas.core.frame.DataFrame'>
Index: 2808 entries, 0 to 5260
Data columns (total 18 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ID                                                      2808 non-null   int64  
 1   First Name                                              2808 non-null   object 
 2   Email                                                   2808 non-null   object 
 3   Gender                                                  115 non-null    object 
 4   City                                                    54 non-null     object 
 5   Created                                                 2808 non-null   object 
 6   Position                                                1 non-null      object 
 7   New College Name                                        831 non-null    object 
 8   Colleges                 

Feature Engineering

In [26]:
def extract_year(created_str):
    created_date = datetime.strptime(created_str, '%m/%d/%Y %I:%M:%S %p')
    return created_date.year

# Apply the function to the 'Created' column and create a new column 'Year'
df['Year'] = df['Created'].apply(extract_year)

Calculate Predicted Graduation Year

In [27]:
def calculate_graduation_year(row):
    """Calculate the predicted graduation year based on academic year."""
    academic_year = row['Academic Year']
    current_year = row['Year']
    if academic_year == 1:
        predicted_year = current_year + 3
        context = 'Predicted based on standard 4-year course.'
    elif academic_year == 2:
        predicted_year = current_year + 2
        context = 'Predicted based on standard 3-year course.'
    elif academic_year == 3:
        if current_year == datetime.now().year:
            predicted_year = current_year + 1
            context = 'Predicted based on current academic year and current date.'
        else:
            predicted_year = current_year + 2
            context = 'Predicted based on standard 4-year course.'
    elif academic_year == 4:
        if current_year == datetime.now().year:
            predicted_year = current_year
            context = 'Predicted based on current academic year and current date.'
        else:
            predicted_year = current_year + 1
            context = 'Predicted based on standard 4-year course.'
    else:
        predicted_year = pd.NA
        context = 'Unable to predict graduation year due to invalid academic year.'
    return predicted_year, context

# Apply the function to create new columns 'Predicted Graduation Year' and 'Context'
df['Predicted Graduation Year'], df['Context'] = zip(*df.apply(calculate_graduation_year, axis=1))

Save Results

In [29]:
output = df[['ID', 'First Name', 'Email', 'New College Name', 'Predicted Graduation Year', 'Context']]
output_file_path = "Predicted_Graduation_with_Context.xlsx"
output.to_excel(output_file_path, index=False, header=True)